In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [36]:
data=pd.read_csv('C:\\Users\\sachi\\OneDrive\\Desktop\\DataInsight.ai\\data\\raw\\laptop_data.csv')

In [37]:
data.head()

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [38]:
data.isnull().sum()

Unnamed: 0          0
Company             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price               0
dtype: int64

In [39]:
data['Ram'] = data['Ram'].str.replace('GB', '').astype('int32')
data['Weight'] = data['Weight'].str.replace('kg', '').astype('float32')
data.head()

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,71378.6832
1,1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,47895.5232
2,2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,30636.0000
3,3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,135195.3360
4,4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,96095.8080


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   TypeName          1303 non-null   object 
 3   Inches            1303 non-null   float64
 4   ScreenResolution  1303 non-null   object 
 5   Cpu               1303 non-null   object 
 6   Ram               1303 non-null   int32  
 7   Memory            1303 non-null   object 
 8   Gpu               1303 non-null   object 
 9   OpSys             1303 non-null   object 
 10  Weight            1303 non-null   float32
 11  Price             1303 non-null   float64
dtypes: float32(1), float64(2), int32(1), int64(1), object(7)
memory usage: 112.1+ KB


In [41]:
# ScreenResolution Feature Engineering
data['Touchscreen'] = data['ScreenResolution'].apply(lambda x: 1 if 'Touchscreen' in x else 0)
data['Ips'] = data['ScreenResolution'].apply(lambda x: 1 if 'IPS' in x else 0)

new = data['ScreenResolution'].str.split('x', n=1, expand=True)
data['X_res'] = new[0]
data['Y_res'] = new[1]
data['X_res'] = data['X_res'].str.replace(',', '').str.findall(r'(\d+\.?\d+)').apply(lambda x: x[0])

data['X_res'] = data['X_res'].astype('int')
data['Y_res'] = data['Y_res'].astype('int')


In [42]:
# Cpu Feature Engineering
data['Cpu Name'] = data['Cpu'].apply(lambda x: " ".join(x.split()[0:3]))

def fetch_processor(text):
    if text == 'Intel Core i7' or text == 'Intel Core i5' or text == 'Intel Core i3':
        return text
    else:
        if text.split()[0] == 'Intel':
            return 'Other Intel Processor'
        else:
            return 'AMD Processor'

data['Cpu brand'] = data['Cpu Name'].apply(fetch_processor)


In [43]:
# Memory Feature Engineering
data['Memory'] = data['Memory'].astype(str).replace('\\.0', '', regex=True)
data['Memory'] = data['Memory'].str.replace('GB', '')
data['Memory'] = data['Memory'].str.replace('TB', '000')
new = data['Memory'].str.split("+", n=1, expand=True)

data['first'] = new[0]
data['first'] = data['first'].str.strip()

data['second'] = new[1]

data['Layer1HDD'] = data['first'].apply(lambda x: 1 if "HDD" in x else 0)
data['Layer1SSD'] = data['first'].apply(lambda x: 1 if "SSD" in x else 0)
data['Layer1Hybrid'] = data['first'].apply(lambda x: 1 if "Hybrid" in x else 0)
data['Layer1Flash_Storage'] = data['first'].apply(lambda x: 1 if "Flash Storage" in x else 0)

data['first'] = data['first'].str.replace(r'\D', '', regex=True)

data['second'].fillna("0", inplace=True)

data['Layer2HDD'] = data['second'].apply(lambda x: 1 if "HDD" in x else 0)
data['Layer2SSD'] = data['second'].apply(lambda x: 1 if "SSD" in x else 0)
data['Layer2Hybrid'] = data['second'].apply(lambda x: 1 if "Hybrid" in x else 0)
data['Layer2Flash_Storage'] = data['second'].apply(lambda x: 1 if "Flash Storage" in x else 0)

data['second'] = data['second'].str.replace(r'\D', '', regex=True)

data['first'] = data['first'].astype(int)
data['second'] = data['second'].astype(int)

data['HDD'] = (data['first'] * data['Layer1HDD'] + data['second'] * data['Layer2HDD'])
data['SSD'] = (data['first'] * data['Layer1SSD'] + data['second'] * data['Layer2SSD'])
data['Hybrid'] = (data['first'] * data['Layer1Hybrid'] + data['second'] * data['Layer2Hybrid'])
data['Flash_Storage'] = (data['first'] * data['Layer1Flash_Storage'] + data['second'] * data['Layer2Flash_Storage'])

data.drop(columns=['first', 'second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
       'Layer1Flash_Storage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
       'Layer2Flash_Storage'], inplace=True)


C:\Users\sachi\AppData\Local\Temp\ipykernel_21440\791591301.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['second'].fillna("0", inplace=True)


In [44]:
# Gpu Feature Engineering
data['Gpu brand'] = data['Gpu'].apply(lambda x: x.split()[0])
data = data[data['Gpu brand'] != 'ARM']

# Cleanup
data.drop(columns=['Unnamed: 0', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'Cpu Name'], inplace=True, errors='ignore')

# One-Hot Encoding
data = pd.get_dummies(data)
data.head()

,Inches,Ram,Weight,Price,Touchscreen,Ips,X_res,Y_res,HDD,SSD,...,OpSys_Windows 7,OpSys_macOS,Cpu brand_AMD Processor,Cpu brand_Intel Core i3,Cpu brand_Intel Core i5,Cpu brand_Intel Core i7,Cpu brand_Other Intel Processor,Gpu brand_AMD,Gpu brand_Intel,Gpu brand_Nvidia
0,13.3,8,1.37,71378.6832,0,1,2560,1600,0,128,...,False,True,False,False,True,False,False,False,True,False
1,13.3,8,1.34,47895.5232,0,0,1440,900,0,0,...,False,True,False,False,True,False,False,False,True,False
2,15.6,8,1.86,30636.0000,0,0,1920,1080,0,256,...,False,False,False,False,True,False,False,False,True,False
3,15.4,16,1.83,135195.3360,0,1,2880,1800,0,512,...,False,True,False,False,False,True,False,True,False,False
4,13.3,8,1.37,96095.8080,0,1,2560,1600,0,256,...,False,True,False,False,True,False,False,False,True,False


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1302 entries, 0 to 1302
Data columns (total 54 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Inches                           1302 non-null   float64
 1   Ram                              1302 non-null   int32  
 2   Weight                           1302 non-null   float32
 3   Price                            1302 non-null   float64
 4   Touchscreen                      1302 non-null   int64  
 5   Ips                              1302 non-null   int64  
 6   X_res                            1302 non-null   int32  
 7   Y_res                            1302 non-null   int32  
 8   HDD                              1302 non-null   int64  
 9   SSD                              1302 non-null   int64  
 10  Hybrid                           1302 non-null   int64  
 11  Flash_Storage                    1302 non-null   int64  
 12  Company_Acer             

In [46]:
# Separate Target and Features
X = data.drop(columns=['Price'])
y = np.log(data['Price'])

# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=2)

# Verification
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1106, 53)
X_test shape: (196, 53)
y_train shape: (1106,)
y_test shape: (196,)


In [47]:
import os

# Create processed data directory
os.makedirs('data/processed', exist_ok=True)

# Save data
X_train.to_csv('data/processed/X_train.csv', index=False)
X_test.to_csv('data/processed/X_test.csv', index=False)
y_train.to_csv('data/processed/y_train.csv', index=False)
y_test.to_csv('data/processed/y_test.csv', index=False)

print("Processed data saved to data/processed/")


Processed data saved to data/processed/
